In [1]:
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

No mac pode haver um erro sobre o arquivo "chromedriver" não ser confiável. Workaround: ir até a pasta onde o arquivo está localizado, clicar com o botão direito e inicializar com o terminal. Após isso, rodar o chunk abaixo.

In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome("../driver/chromedriver", options=options)

In [3]:
def go_to_website(website, driver):
    driver.get(website) 
    return None

In [4]:
go_to_website('https://br.indeed.com/', driver)

# Parte 1: Listar vagas

Pesquisar vaga

In [5]:
def aceitar_coookies(driver):
    try:
        driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass
    return None

aceitar_coookies(driver)

In [6]:
def pesquisar_qual_vaga(palavra_chave, driver):
    vaga = driver.find_element_by_name('q') 
    vaga.clear() 
    vaga.send_keys(palavra_chave)
    vaga.send_keys(Keys.RETURN)
    return None

pesquisar_qual_vaga("Cientista de Dados", driver)

In [7]:
def pesquisar_local_vaga(palavra_chave, driver):
    cargo_input = driver.find_element_by_name('l') 
    cargo_input.send_keys(Keys.COMMAND + "a")
    cargo_input.send_keys(Keys.DELETE)
    cargo_input.send_keys(palavra_chave)
    cargo_input.send_keys(Keys.RETURN)
    return None

pesquisar_local_vaga("Brasil", driver)

In [8]:
def fechar_popup_logar_com_google(driver):
    try:
        driver.find_elements_by_css_selector("[aria-label=Close]")[1].click()
        sleep(2)
        driver.find_elements_by_css_selector("[aria-label=fechar]")[1].click()
    except:
        pass
    return None

fechar_popup_logar_com_google(driver)

In [9]:
def fechar_popup_propaganda(driver):
    try:
        driver.find_elements_by_css_selector("[aria-label='Close job details']")[0].click()
    except:
        pass
    return None

fechar_popup_propaganda(driver)

Listar vagas

In [10]:
def build_df_vagas(driver):
    
    lista_vagas = driver.find_elements_by_xpath('//div[@id="mosaic-provider-jobcards"]//ul[@class="jobsearch-ResultsList css-0"]//h2[@class = "jobTitle css-1h4a4n5 eu4oa1w0"]//a')
    
    D = []
    for vaga in lista_vagas:

        d = {
          "id": vaga.get_attribute("id"),
          "vaga": vaga.get_attribute("text"),
          "url": vaga.get_attribute("href")
        }
        D.append(d)
    df = pd.DataFrame.from_dict(D)
    return df

df = build_df_vagas(driver)

df

,id,vaga,url
0,job_637d9d8bc023151e,Tecnologia | Cientista de Dados e Analytics,https://br.indeed.com/rc/clk?jk=637d9d8bc02315...
1,job_aed977c486141ccd,Cientista de Dados - Junior (Remoto),https://br.indeed.com/rc/clk?jk=aed977c486141c...
2,job_c01a031030544c68,CIENTISTA DE DADOS JUNIOR,https://br.indeed.com/rc/clk?jk=c01a031030544c...
3,job_52c176f56dcce125,Cientista de Dados Jr,https://br.indeed.com/rc/clk?jk=52c176f56dcce1...
4,job_b5177dcf82d997a1,Cientista de Dados Junior,https://br.indeed.com/rc/clk?jk=b5177dcf82d997...
5,job_349178cd7b1aaa05,Cientista de Dados,https://br.indeed.com/rc/clk?jk=349178cd7b1aaa...
6,job_eaf5f6322ee0b529,Cientista de Dados,https://br.indeed.com/company/4yousee/jobs/Cie...
7,job_0a789d69aa06bcd2,Cientista de Dados,https://br.indeed.com/rc/clk?jk=0a789d69aa06bc...
8,job_8e85d1399ed0777d,Pessoa Especialista de Dados,https://br.indeed.com/rc/clk?jk=8e85d1399ed077...
9,job_e67a187cb0746190,Cientista de Dados,https://br.indeed.com/rc/clk?jk=e67a187cb07461...


Ir para próxima página

In [11]:
def go_to_next_page(driver):
    
    url = driver.find_elements_by_xpath('//a[@data-testid="pagination-page-next"]')[0].get_attribute('href')
    
    go_to_website(url, driver)
    
    return None

go_to_next_page(driver)

# Parte 2: Extrair dados de cada vaga

Ir para a vaga

In [12]:
urls = df.url.to_list()

In [13]:
go_to_website(urls[3], driver)

In [14]:
def get_titulo_vaga(driver):
    titulo = driver.find_elements_by_xpath('//div[@class="jobsearch-DesktopStickyContainer"]//h1')[0].text
    return titulo
get_titulo_vaga(driver)

'Cientista de Dados Jr'

In [15]:
def get_info_vaga(driver):
    info_vaga = driver.find_elements_by_xpath('//div[@class="jobsearch-DesktopStickyContainer"]//div[@class="jobsearch-CompanyInfoContainer"]')
    info_vaga = info_vaga[0].text.split("\n")
    info_vaga = ";".join(info_vaga)
    return info_vaga

get_info_vaga(driver)

'Fretebras;Remoto;Home office'

In [16]:
def get_descricao_da_vaga(driver):
    desc = driver.find_elements_by_xpath('//div[@id="jobDescriptionText"]')[0].text.split("\n")
    desc = list(filter(None, desc))
    desc = ";".join(desc)
    return desc

get_descricao_da_vaga(driver)

'#VemSerFrete;A Fretebras é a pioneira e a maior plataforma tecnológica para o transporte de cargas da América do Sul. A empresa conecta transportadoras e caminhoneiros, investindo em soluções inovadoras para tornar o transporte rodoviário mais eficiente.;Somos um time que trabalha para transformar o nosso transporte e construir processos mais eficientes entre tecnologia, inteligência artificial e pessoas. Nos dedicamos a melhorar a experiência e simplificar a vida dos caminhoneiros e dos nossos clientes. Nos organizamos em squads, utilizamos metodologias ágeis, somos informais, colaborativos, valorizamos muito a empatia e o trabalho em equipe.;Responsabilidades e atribuições;Analisar os dados, questionando suas observações, levantando novas hipóteses e buscando gerar inteligência para suportar nossas decisões de negócio;;Compartilhar com o time e com a empresa os resultados encontrados nas análises, tenham eles provado ou não a hipótese inicial;;Construir modelos estatísticos e de Mac

In [17]:
def build_dic_info_vagas(url, driver):
    
    go_to_website(url, driver)
    
    d = {
      "titulo": get_titulo_vaga(driver),
      "info": get_info_vaga(driver),
      "descricao": get_descricao_da_vaga(driver)
    }

    return d
build_dic_info_vagas(urls[0], driver)

{'titulo': 'Tecnologia | Cientista de Dados e Analytics',
 'info': 'Cielo;227 avaliações;Remoto;Home office',
 'descricao': 'Se você gosta de possibilidades de desenvolvimento pessoal e profissional, além de simplificar e impulsionar negócios através de oportunidades na empresa líder de segmento de pagamentos eletrônicos na América Latina, seu lugar é junto ao #TimeCielo!;Infinitas possibilidades <3;Aqui na Cielo acreditamos que nenhum negócio nasceu para ficar parado e, entendemos que a nossa vocação é despertar essa mesa inquietude em cada um dos nossos clientes. Ao todo, somos mais 4 mil pessoas colaboradoras que simplificam e impulsionam negócios para todos, afinal, não é à toa que somos uma empresa de tecnologia e serviço para o varejo, líder no segmento de pagamentos eletrônicos na América Latina.;Oferecemos um portfólio de soluções para atender às necessidades de mais de 1,2 milhão de clientes, desde os pequenos lojistas até os grandes varejistas espalhados por todo o país.;Qual

In [18]:
D = []
for url in urls:
    d = build_dic_info_vagas(url, driver)
    D.append(d)
    sleep(1)
    
df_infos = pd.DataFrame.from_dict(D)

In [19]:
df_infos

,titulo,info,descricao
0,Tecnologia | Cientista de Dados e Analytics,Cielo;227 avaliações;Remoto;Home office,Se você gosta de possibilidades de desenvolvim...
1,Cientista de Dados - Junior (Remoto),BIX Tecnologia;Remoto;Home office,"Somos uma empresa apaixonada por pessoas, paut..."
2,CIENTISTA DE DADOS JUNIOR,"BRT INTELLIGENCE;15 avaliações;Belo Horizonte,...",Venha fazer parte do #JeitodeSerBRT!;A BRT Int...
3,Cientista de Dados Jr,Fretebras;Remoto;Home office,#VemSerFrete;A Fretebras é a pioneira e a maio...
4,Cientista de Dados Junior,"C6 Bank;8 avaliações;São Paulo, SP","O banco da sua vida;Somos um banco completo, l..."
5,Cientista de Dados,"Takao;São Paulo, SP;Home office",Já pensou em trabalhar em um lugar em constant...
6,Cientista de Dados,"4yousee;Belo Horizonte, MG;Home office",Requisitos:;Conhecimentos sobre análises de da...
7,Cientista de Dados,A3Data;Remoto;Home office,A A3Data é uma consultoria referência em #Dado...
8,Pessoa Especialista de Dados,Nova Escola;6 avaliações;Remoto;Home office,"Olá, ficamos muito felizes com seu interesse!;..."
9,Cientista de Dados,"Segware;2 avaliações;Campinas, SP;Home office",Cientista de Dados;Estamos buscando Cientistas...


In [20]:
df_completo = df.join(df_infos)
df_completo

,id,vaga,url,titulo,info,descricao
0,job_637d9d8bc023151e,Tecnologia | Cientista de Dados e Analytics,https://br.indeed.com/rc/clk?jk=637d9d8bc02315...,Tecnologia | Cientista de Dados e Analytics,Cielo;227 avaliações;Remoto;Home office,Se você gosta de possibilidades de desenvolvim...
1,job_aed977c486141ccd,Cientista de Dados - Junior (Remoto),https://br.indeed.com/rc/clk?jk=aed977c486141c...,Cientista de Dados - Junior (Remoto),BIX Tecnologia;Remoto;Home office,"Somos uma empresa apaixonada por pessoas, paut..."
2,job_c01a031030544c68,CIENTISTA DE DADOS JUNIOR,https://br.indeed.com/rc/clk?jk=c01a031030544c...,CIENTISTA DE DADOS JUNIOR,"BRT INTELLIGENCE;15 avaliações;Belo Horizonte,...",Venha fazer parte do #JeitodeSerBRT!;A BRT Int...
3,job_52c176f56dcce125,Cientista de Dados Jr,https://br.indeed.com/rc/clk?jk=52c176f56dcce1...,Cientista de Dados Jr,Fretebras;Remoto;Home office,#VemSerFrete;A Fretebras é a pioneira e a maio...
4,job_b5177dcf82d997a1,Cientista de Dados Junior,https://br.indeed.com/rc/clk?jk=b5177dcf82d997...,Cientista de Dados Junior,"C6 Bank;8 avaliações;São Paulo, SP","O banco da sua vida;Somos um banco completo, l..."
5,job_349178cd7b1aaa05,Cientista de Dados,https://br.indeed.com/rc/clk?jk=349178cd7b1aaa...,Cientista de Dados,"Takao;São Paulo, SP;Home office",Já pensou em trabalhar em um lugar em constant...
6,job_eaf5f6322ee0b529,Cientista de Dados,https://br.indeed.com/company/4yousee/jobs/Cie...,Cientista de Dados,"4yousee;Belo Horizonte, MG;Home office",Requisitos:;Conhecimentos sobre análises de da...
7,job_0a789d69aa06bcd2,Cientista de Dados,https://br.indeed.com/rc/clk?jk=0a789d69aa06bc...,Cientista de Dados,A3Data;Remoto;Home office,A A3Data é uma consultoria referência em #Dado...
8,job_8e85d1399ed0777d,Pessoa Especialista de Dados,https://br.indeed.com/rc/clk?jk=8e85d1399ed077...,Pessoa Especialista de Dados,Nova Escola;6 avaliações;Remoto;Home office,"Olá, ficamos muito felizes com seu interesse!;..."
9,job_e67a187cb0746190,Cientista de Dados,https://br.indeed.com/rc/clk?jk=e67a187cb07461...,Cientista de Dados,"Segware;2 avaliações;Campinas, SP;Home office",Cientista de Dados;Estamos buscando Cientistas...
